<a href="https://colab.research.google.com/github/PhoenixPlanet/Hongong-Machine/blob/main/ch_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>